In [1]:
# Đọc file paraquet 
import pandas as pd

df = pd.read_parquet("/Users/buu_os/Library/CloudStorage/OneDrive-Personal/Test project/Database/Buu_clean_ver2.parquet", engine="pyarrow")
# Đọc 5 dòng đầu tiên
print(df.head())


  REPORT_DATE  YEAR  QUARTER FREQ_CODE SECURITY_CODE AUDITED METRIC_CODE  \
0  2018-12-31  2018        4         Q           TPB       N     BBS_100   
1  2018-12-31  2018        4         Y           TPB       Y     BBS_100   
2  2018-12-31  2018        4         Q           NVB       N     BBS_100   
3  2018-12-31  2018        4         Y           NVB       Y     BBS_100   
4  2018-12-31  2018        4         Q           VPB       N     BBS_100   

        Mô tả  METRIC_VALUE     ICB_L2  
0  A. Tài sản    136179.403  Ngân hàng  
1  A. Tài sản    136179.403  Ngân hàng  
2  A. Tài sản     72431.770  Ngân hàng  
3  A. Tài sản     72422.170  Ngân hàng  
4  A. Tài sản    323308.442  Ngân hàng  


In [2]:
# Hiện thị số lượng tất cả Ticker trong file Buu_clean_ver2.parquet và thống kê cho tôi các mã chia theo từng ngành tỷ trọng như thế nào 
import pandas as pd

df = pd.read_parquet("/Users/buu_os/Library/CloudStorage/OneDrive-Personal/Test project/Database/Buu_clean_ver2.parquet", engine="pyarrow")

# Hiển thị số lượng mã cổ phiếu (ticker) duy nhất trong database
num_tickers = df['SECURITY_CODE'].nunique()
print(f"Số lượng mã cổ phiếu trong database: {num_tickers}")

# Thống kê số lượng cổ phiếu (SECURITY_CODE) duy nhất theo từng ngành (ICB_L2)
ticker_per_sector = df.groupby('ICB_L2')['SECURITY_CODE'].nunique().sort_values(ascending=False)
print("Số lượng cổ phiếu theo từng ngành:")
print(ticker_per_sector)

# Hiển thị tỷ lệ các mã cổ phiếu theo từng ngành theo tổng số mã cổ phiếu (%)
print(df['ICB_L2'].value_counts(normalize=True) * 100)  

Số lượng mã cổ phiếu trong database: 457
Số lượng cổ phiếu theo từng ngành:
ICB_L2
Xây dựng và Vật liệu             76
Bất động sản                     75
Thực phẩm và đồ uống             39
Hàng & Dịch vụ Công nghiệp       38
Tài nguyên Cơ bản                37
Dịch vụ tài chính                37
Hóa chất                         27
Ngân hàng                        24
Điện, nước & xăng dầu khí đốt    18
Hàng cá nhân & Gia dụng          14
Y tế                             14
Du lịch và Giải trí              10
Bán lẻ                            9
Dầu khí                           8
Công nghệ Thông tin               8
Ô tô và phụ tùng                  7
Bảo hiểm                          6
Truyền thông                      6
Viễn thông                        4
Name: SECURITY_CODE, dtype: int64
ICB_L2
Dịch vụ tài chính                19.110138
Bất động sản                     14.624497
Xây dựng và Vật liệu             13.835475
Thực phẩm và đồ uống              7.211661
Hàng & Dịch vụ Công 

In [32]:
# Tính toán lợi nhuận hoạt động MWG: CIS_20 + CIS_25 + CIS_26 theo năm (FREQ_CODE='Y')
import pandas as pd

df = pd.read_parquet("/Users/buu_os/Library/CloudStorage/OneDrive-Personal/Test project/Database/Buu_clean_ver2.parquet", engine="pyarrow")
df_mwg = df[(df['SECURITY_CODE'] == 'MWG') & (df['FREQ_CODE'] == 'Y')]

print(f"Số bản ghi: {len(df_mwg)} | Năm: {sorted(df_mwg['YEAR'].unique())}")

# Pivot để lấy từng metric thành 1 cột
pivot = df_mwg.pivot(index='YEAR', columns='METRIC_CODE', values='METRIC_VALUE')
pivot = pivot[['CIS_20', 'CIS_25', 'CIS_26']]  # Đảm bảo đúng thứ tự

# Tính lợi nhuận hoạt động: CIS_20 + CIS_25 + CIS_26
pivot['OPERATING_PROFIT'] = pivot['CIS_20'] + pivot['CIS_25'] + pivot['CIS_26']

print("Lợi nhuận hoạt động theo năm:")
print(pivot[['OPERATING_PROFIT']])

# Tính tăng trưởng
pivot['REVENUE_GROWTH'] = pivot['CIS_20'].pct_change()
pivot['OPERATING_PROFIT_GROWTH'] = pivot['OPERATING_PROFIT'].pct_change()
print("\nTăng trưởng doanh thu và LN hoạt động (%):")
print(pivot[['REVENUE_GROWTH', 'OPERATING_PROFIT_GROWTH']])

Số bản ghi: 3416 | Năm: [2018, 2019, 2020, 2021, 2022, 2023, 2024]
Lợi nhuận hoạt động theo năm:
METRIC_CODE  OPERATING_PROFIT
YEAR                         
2018              3870.772674
2019              4976.733705
2020              5216.266604
2021              5894.568618
2022              6644.176849
2023               436.250746
2024              4083.604884

Tăng trưởng doanh thu và LN hoạt động (%):
METRIC_CODE  REVENUE_GROWTH  OPERATING_PROFIT_GROWTH
YEAR                                                
2018                    NaN                      NaN
2019               0.274368                 0.285721
2020               0.229205                 0.048131
2021               0.153526                 0.130036
2022               0.116890                 0.127169
2023              -0.270281                -0.934341
2024               0.221068                 8.360683


In [29]:
# Tính toán lợi nhuận hoạt động bằng cách lấy CIS_20 - CIS_25 - CIS_26 cho mã MWG theo báo cáo năm (FREQ_CODE='Y')
import pandas as pd

df = pd.read_parquet("/Users/buu_os/Library/CloudStorage/OneDrive-Personal/Test project/Database/Buu_clean_ver2.parquet", engine="pyarrow")

# Lọc dữ liệu cho mã MWG và báo cáo năm
df_mwg_yearly = df[(df['SECURITY_CODE'] == 'MWG') & (df['FREQ_CODE'] == 'Y')].copy()

print("Dữ liệu MWG theo năm:")
print(f"Số lượng bản ghi: {len(df_mwg_yearly)}")
print(f"Các năm có dữ liệu: {sorted(df_mwg_yearly['YEAR'].unique())}")
print("\n" + "="*50 + "\n")

# Lọc các metric cần thiết: CIS_20, CIS_25, CIS_26
# Chuẩn hóa tên cột: REVENUE, SELLING_EXPENSE, ADMIN_EXPENSE, DESC
cis_20_data = df_mwg_yearly[df_mwg_yearly['METRIC_CODE'] == 'CIS_20'][['YEAR', 'METRIC_VALUE', 'Mô tả']].rename(
    columns={'METRIC_VALUE': 'REVENUE', 'Mô tả': 'REVENUE_DESC'}
)
cis_25_data = df_mwg_yearly[df_mwg_yearly['METRIC_CODE'] == 'CIS_25'][['YEAR', 'METRIC_VALUE', 'Mô tả']].rename(
    columns={'METRIC_VALUE': 'SELLING_EXPENSE', 'Mô tả': 'SELLING_EXPENSE_DESC'}
)
cis_26_data = df_mwg_yearly[df_mwg_yearly['METRIC_CODE'] == 'CIS_26'][['YEAR', 'METRIC_VALUE', 'Mô tả']].rename(
    columns={'METRIC_VALUE': 'ADMIN_EXPENSE', 'Mô tả': 'ADMIN_EXPENSE_DESC'}
)

print("Dữ liệu CIS_20 (Doanh thu thuần):")
print(cis_20_data)
print("\n" + "="*50 + "\n")

print("Dữ liệu CIS_25 (Chi phí bán hàng):")
print(cis_25_data)
print("\n" + "="*50 + "\n")

print("Dữ liệu CIS_26 (Chi phí quản lý và doanh nghiệp):")
print(cis_26_data)
print("\n" + "="*50 + "\n")

# Merge dữ liệu theo năm
operating_profit_df = cis_20_data.merge(cis_25_data, on='YEAR')
operating_profit_df = operating_profit_df.merge(cis_26_data, on='YEAR')

# Tính toán lợi nhuận hoạt động: REVENUE - SELLING_EXPENSE - ADMIN_EXPENSE
operating_profit_df['OPERATING_PROFIT'] = (
    operating_profit_df['REVENUE']
    + operating_profit_df['SELLING_EXPENSE']
    + operating_profit_df['ADMIN_EXPENSE']
)

print("Kết quả tính toán lợi nhuận hoạt động:")
print(operating_profit_df[['YEAR', 'REVENUE', 'SELLING_EXPENSE', 'ADMIN_EXPENSE', 'OPERATING_PROFIT']])

print("\n" + "="*50 + "\n")
print("Mô tả các chỉ số:")
print(f"CIS_20: {operating_profit_df['REVENUE_DESC'].iloc[0]}")
print(f"CIS_25: {operating_profit_df['SELLING_EXPENSE_DESC'].iloc[0]}")
print(f"CIS_26: {operating_profit_df['ADMIN_EXPENSE_DESC'].iloc[0]}")

# Tính toán tăng trưởng
operating_profit_df['REVENUE_GROWTH'] = operating_profit_df['REVENUE'].pct_change()
operating_profit_df['OPERATING_PROFIT_GROWTH'] = operating_profit_df['OPERATING_PROFIT'].pct_change()

print("\n" + "="*50 + "\n")
print("Lợi nhuận hoạt động theo năm:")
print(operating_profit_df[['YEAR', 'OPERATING_PROFIT']])

Dữ liệu MWG theo năm:
Số lượng bản ghi: 3416
Các năm có dữ liệu: [2018, 2019, 2020, 2021, 2022, 2023, 2024]


Dữ liệu CIS_20 (Doanh thu thuần):
         YEAR       REVENUE                                       REVENUE_DESC
3416948  2019  19487.799304  5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...
3417234  2020  23954.497272  5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...
3419028  2018  15292.127763  5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...
3421465  2022  30862.042496  5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...
3423065  2021  27632.131996  5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...
3425855  2023  22520.613164  5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...
3429207  2024  27499.210077  5. Lợi nhuận gộp về bán hàng và cung cấp dịch ...


Dữ liệu CIS_25 (Chi phí bán hàng):
         YEAR  SELLING_EXPENSE SELLING_EXPENSE_DESC
3499138  2019    -12437.282718  9. Chi phí bán hàng
3499424  2020    -15333.798831  9. Chi phí bán hàng
3501218  2018     -9659.741189 

In [45]:
# Tính toán lợi nhuận hoạt động cho toàn bộ ticker và xuất ra file CSV
import pandas as pd

# Đọc dữ liệu Parquet
df = pd.read_parquet("/Users/buu_os/Library/CloudStorage/OneDrive-Personal/Test project/Database/Buu_clean_ver2.parquet", engine="pyarrow")

# Lọc chỉ báo cần thiết và báo cáo năm
metrics = ['CIS_20', 'CIS_25', 'CIS_26']
df_year = df[(df['FREQ_CODE'] == 'Y') & (df['METRIC_CODE'].isin(metrics))]

# Pivot để mỗi ticker-năm là 1 dòng, mỗi metric là 1 cột
pivot = df_year.pivot_table(
    index=['SECURITY_CODE', 'YEAR'],
    columns='METRIC_CODE',
    values='METRIC_VALUE'
).reset_index()

# Đảm bảo đủ cột, nếu thiếu sẽ là NaN
for col in metrics:
    if col not in pivot.columns:
        pivot[col] = pd.NA

# Đổi tên cột cho dễ hiểu
pivot = pivot.rename(columns={
    'CIS_20': 'REVENUE',
    'CIS_25': 'SELLING_EXPENSE',
    'CIS_26': 'ADMIN_EXPENSE'
})

# Tính lợi nhuận hoạt động: REVENUE + SELLING_EXPENSE + ADMIN_EXPENSE
pivot['OPERATING_PROFIT'] = (
    pivot['REVENUE'].fillna(0)
    + pivot['SELLING_EXPENSE'].fillna(0)
    + pivot['ADMIN_EXPENSE'].fillna(0)
)

# Sắp xếp để tính tăng trưởng
pivot = pivot.sort_values(['SECURITY_CODE', 'YEAR']).reset_index(drop=True)

# Tính tăng trưởng doanh thu và lợi nhuận hoạt động theo từng ticker
pivot['REVENUE_GROWTH'] = pivot.groupby('SECURITY_CODE')['REVENUE'].pct_change()
pivot['OPERATING_PROFIT_GROWTH'] = pivot.groupby('SECURITY_CODE')['OPERATING_PROFIT'].pct_change()

# Xuất ra file CSV
output_path = "operating_profit_all_tickers.csv"
pivot.to_csv(output_path, index=False)
print(f"Đã xuất kết quả ra file: {output_path}")

# Hiển thị 10 dòng đầu để kiểm tra
print(pivot.head(10)[['SECURITY_CODE', 'YEAR', 'REVENUE', 'SELLING_EXPENSE', 'ADMIN_EXPENSE', 'OPERATING_PROFIT', 'REVENUE_GROWTH', 'OPERATING_PROFIT_GROWTH']])

#làm filter hiển thị 10 dòng đầu tiên của file csv
df = pd.read_csv("operating_profit_all_tickers.csv")
print(df.head(10))

Đã xuất kết quả ra file: operating_profit_all_tickers.csv
METRIC_CODE SECURITY_CODE  YEAR      REVENUE  SELLING_EXPENSE  ADMIN_EXPENSE  \
0                     AAA  2018   672.936073      -156.562810    -123.618312   
1                     AAA  2019  1042.139379      -214.621335    -149.020821   
2                     AAA  2020   776.935221      -255.844520    -166.965893   
3                     AAA  2021  1301.375383      -738.538105    -194.751557   
4                     AAA  2022  1086.239884      -666.097261    -247.607550   
5                     AAA  2023  1108.693323      -511.055879    -285.281847   
6                     AAA  2024  1484.105506      -716.779456    -380.383295   
7                     AAH  2020    32.206856        -3.261071     -64.233630   
8                     AAH  2021    34.190018        -3.990215     -20.660320   
9                     AAH  2022   151.627348        -9.520645     -23.538521   

METRIC_CODE  OPERATING_PROFIT  REVENUE_GROWTH  OPERATING_PROF

In [46]:
# Tính toán lợi nhuận hoạt động cho toàn bộ ticker và xuất ra file CSV (năm và quý)
import pandas as pd

# Đọc dữ liệu Parquet
df = pd.read_parquet("/Users/buu_os/Library/CloudStorage/OneDrive-Personal/Test project/Database/Buu_clean_ver2.parquet", engine="pyarrow")

metrics = ['CIS_20', 'CIS_25', 'CIS_26']

# =========================
# PHIÊN BẢN THEO NĂM (Y)
# =========================
df_year = df[(df['FREQ_CODE'] == 'Y') & (df['METRIC_CODE'].isin(metrics))]

pivot_year = df_year.pivot_table(
    index=['SECURITY_CODE', 'YEAR'],
    columns='METRIC_CODE',
    values='METRIC_VALUE'
).reset_index()

for col in metrics:
    if col not in pivot_year.columns:
        pivot_year[col] = pd.NA

pivot_year = pivot_year.rename(columns={
    'CIS_20': 'REVENUE',
    'CIS_25': 'SELLING_EXPENSE',
    'CIS_26': 'ADMIN_EXPENSE'
})

pivot_year['OPERATING_PROFIT'] = (
    pivot_year['REVENUE'].fillna(0)
    + pivot_year['SELLING_EXPENSE'].fillna(0)
    + pivot_year['ADMIN_EXPENSE'].fillna(0)
)

pivot_year = pivot_year.sort_values(['SECURITY_CODE', 'YEAR']).reset_index(drop=True)
pivot_year['REVENUE_GROWTH'] = pivot_year.groupby('SECURITY_CODE')['REVENUE'].pct_change()
pivot_year['OPERATING_PROFIT_GROWTH'] = pivot_year.groupby('SECURITY_CODE')['OPERATING_PROFIT'].pct_change()

output_path_year = "operating_profit_all_tickers.csv"
pivot_year.to_csv(output_path_year, index=False)
print(f"Đã xuất kết quả ra file: {output_path_year}")
print(pivot_year.head(10)[['SECURITY_CODE', 'YEAR', 'REVENUE', 'SELLING_EXPENSE', 'ADMIN_EXPENSE', 'OPERATING_PROFIT', 'REVENUE_GROWTH', 'OPERATING_PROFIT_GROWTH']])



Đã xuất kết quả ra file: operating_profit_all_tickers.csv
METRIC_CODE SECURITY_CODE  YEAR      REVENUE  SELLING_EXPENSE  ADMIN_EXPENSE  \
0                     AAA  2018   672.936073      -156.562810    -123.618312   
1                     AAA  2019  1042.139379      -214.621335    -149.020821   
2                     AAA  2020   776.935221      -255.844520    -166.965893   
3                     AAA  2021  1301.375383      -738.538105    -194.751557   
4                     AAA  2022  1086.239884      -666.097261    -247.607550   
5                     AAA  2023  1108.693323      -511.055879    -285.281847   
6                     AAA  2024  1484.105506      -716.779456    -380.383295   
7                     AAH  2020    32.206856        -3.261071     -64.233630   
8                     AAH  2021    34.190018        -3.990215     -20.660320   
9                     AAH  2022   151.627348        -9.520645     -23.538521   

METRIC_CODE  OPERATING_PROFIT  REVENUE_GROWTH  OPERATING_PROF

In [47]:
# =========================
# PHIÊN BẢN THEO QUÝ (Q)
# =========================
# Chỉ lấy dữ liệu quý, loại bỏ báo cáo năm
df_quarter = df[(df['FREQ_CODE'] == 'Q') & (df['METRIC_CODE'].isin(metrics))]

# Lọc theo khoảng thời gian 2018-Q1 đến 2025-Q2
# Giả sử cột YEAR là int, QUARTER là int (1-4)
mask_time = (
    ((df_quarter['YEAR'] > 2018) & (df_quarter['YEAR'] < 2025)) |
    ((df_quarter['YEAR'] == 2018) & (df_quarter['QUARTER'] >= 1)) |
    ((df_quarter['YEAR'] == 2025) & (df_quarter['QUARTER'] <= 2))
)
df_quarter = df_quarter[mask_time]

# Pivot: mỗi ticker-quý là 1 dòng, mỗi metric là 1 cột
pivot_quarter = df_quarter.pivot_table(
    index=['SECURITY_CODE', 'YEAR', 'QUARTER'],
    columns='METRIC_CODE',
    values='METRIC_VALUE'
).reset_index()

for col in metrics:
    if col not in pivot_quarter.columns:
        pivot_quarter[col] = pd.NA

pivot_quarter = pivot_quarter.rename(columns={
    'CIS_20': 'REVENUE',
    'CIS_25': 'SELLING_EXPENSE',
    'CIS_26': 'ADMIN_EXPENSE'
})

pivot_quarter['OPERATING_PROFIT'] = (
    pivot_quarter['REVENUE'].fillna(0)
    + pivot_quarter['SELLING_EXPENSE'].fillna(0)
    + pivot_quarter['ADMIN_EXPENSE'].fillna(0)
)

pivot_quarter = pivot_quarter.sort_values(['SECURITY_CODE', 'YEAR', 'QUARTER']).reset_index(drop=True)
pivot_quarter['REVENUE_GROWTH'] = pivot_quarter.groupby('SECURITY_CODE')['REVENUE'].pct_change()
pivot_quarter['OPERATING_PROFIT_GROWTH'] = pivot_quarter.groupby('SECURITY_CODE')['OPERATING_PROFIT'].pct_change()

# Xuất ra file CSV chỉ gồm quý (không gồm báo cáo năm)
output_path_quarter = "operating_profit_all_tickers_quarter.csv"
pivot_quarter.to_csv(output_path_quarter, index=False)
print(f"Đã xuất kết quả ra file: {output_path_quarter}")
print(pivot_quarter.head(10)[['SECURITY_CODE', 'YEAR', 'QUARTER', 'REVENUE', 'SELLING_EXPENSE', 'ADMIN_EXPENSE', 'OPERATING_PROFIT', 'REVENUE_GROWTH', 'OPERATING_PROFIT_GROWTH']])

# Hiển thị 10 dòng đầu tiên của file quý
df_check = pd.read_csv("operating_profit_all_tickers_quarter.csv")
print(df_check.head(10))

Đã xuất kết quả ra file: operating_profit_all_tickers_quarter.csv
METRIC_CODE SECURITY_CODE  YEAR  QUARTER     REVENUE  SELLING_EXPENSE  \
0                     AAA  2018        1  147.525799       -31.202480   
1                     AAA  2018        2  170.103448       -34.097236   
2                     AAA  2018        3  157.839969       -39.008661   
3                     AAA  2018        4  197.466857       -52.254433   
4                     AAA  2019        1  374.652687       -67.805674   
5                     AAA  2019        2  291.414460       -53.853853   
6                     AAA  2019        3  199.576325       -57.806655   
7                     AAA  2019        4  176.495908       -37.062485   
8                     AAA  2020        1  178.765552       -58.328520   
9                     AAA  2020        2  183.365659       -56.663819   

METRIC_CODE  ADMIN_EXPENSE  OPERATING_PROFIT  REVENUE_GROWTH  \
0               -26.960452         89.362868             NaN   
1 